In [173]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing,load_breast_cancer

In [174]:
dta=load_breast_cancer(as_frame=True)
df=pd.DataFrame(dta.frame)
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [175]:
def find_entropy(df):
    
    Class=df.keys()[-1]
    outcomes=df[Class].unique()
    entropy=0
    for outcome in outcomes:
        prob=len(df[Class][df[Class]==outcome])/len(df)
        entropy+= -(prob*np.log2(prob))
    return entropy

In [176]:
find_entropy(df)

0.9526351224018599

In [177]:
def find_avg_entropy(df,attribute):
    Class =df.keys()[-1]
    outcomes=df[Class].unique()
    attribute_values=df[attribute].unique()
    average_entropy=0
    for value in attribute_values:
        entropy_subsample=0
        for value1 in outcomes:
            num=len(df[attribute][df[attribute]==value][df[Class]==value1])
            den=len(df[attribute][df[attribute]==value])
            prob=num/den
            entropy_subsample+= -(prob*np.log2(prob+1e-10))
        weight=den/len(df)
        average_entropy+=weight*entropy_subsample
    return average_entropy

In [178]:
find_avg_entropy(df,'mean smoothness')

0.17645628804335844

In [179]:
def find_winner(df):
    IG=[]
    entropy=find_entropy(df)
    for key in df.keys()[:-1]:
        IG.append(entropy-find_avg_entropy(df,key))
    return df.keys()[:-1][np.argmax(IG)],IG

In [180]:
# find_winner(df)

In [181]:
def subsample(df,attribute,value):
    return df[df[attribute]==value].reset_index(drop=True)

In [182]:
def build_tree(df,tree=None):
    Class=df.keys()[-1]
    attribute=find_winner(df)[0]
    if tree is None:
        tree={}
        tree[attribute]={}
    attr_values=df[attribute].unique()
    for value in attr_values:
        sub=subsample(df,attribute=attribute,value=value)
        ClValue,counts=np.unique(sub[Class],return_counts=True)
        if len(counts)==1:
            tree[attribute][value]=ClValue[0]
        else:
            tree[attribute][value]=build_tree(sub)
    return tree

In [183]:
tree =build_tree(df)

In [ ]:
tree

In [189]:
def predict(instance,tree=None):
    for attribute in tree.keys():
        value=instance[attribute]
        tree=tree[attribute][value]
        if type(tree) ==dict:
            prediction =predict(instance,tree)
        else:
            prediction=tree
    return prediction

In [190]:
i=35
predict(df.iloc[i,:],tree),df.iloc[i,-1]


(0, 0)

In [ ]:
df1=df.sample(100)
Y_lable=[]
for i in range(len(df1)):
    Y_lable.append(predict(df.iloc[i,:],tree))
Y_lable

In [196]:
from sklearn import metrics
metrics.classification_report(df1.iloc[:,-1],Y_lable) 

'              precision    recall  f1-score   support\n\n           0       0.38      0.69      0.50        36\n           1       0.69      0.38      0.48        64\n\n    accuracy                           0.49       100\n   macro avg       0.54      0.53      0.49       100\nweighted avg       0.58      0.49      0.49       100\n'